In [19]:
import pandas as pd
import pdfplumber
import os

# 设置目录路径
directory = '../invoice-xmy/0-unused/'

# 创建空的DataFrame来存储PDF文本信息
df_list = []

# 遍历目录下的所有文件
for filename in os.listdir(directory):
    if filename.lower().endswith('.pdf'):
        # 打开PDF文件
        with pdfplumber.open(os.path.join(directory, filename)) as pdf:
            # 遍历每一页并提取文本信息
            content = ''
            for page in pdf.pages:
                content += page.extract_text()

            # 将文件名和提取出的文本信息添加到列表中
            df_list.append(pd.DataFrame({'filename': [filename], 'content': [content]}))

# 使用concat函数将所有的DataFrame拼接成一个
df = pd.concat(df_list, ignore_index=True)

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)

In [20]:
# 使用replace()函数将所有冒号替换为冒号
df = df.replace({r'：': ':'}, regex=True)
# 使用replace()函数将所有空格替换为空字符串
df['content'] = df['content'].str.replace(' ', '')
df['content'] = df['content'].str.replace('\n', '')

In [21]:
import re

for index, raw in df.iterrows():
    #提取content列的文本内容
    text = raw['content']
    
    print("文件名",raw['filename'])
    
    # 提取发票代码
    #df.loc[index, 'invoice_code'] = re.search(r'发票代码:(\d+)', text).group(1)
    #print("发票代码:", df.loc[index, 'invoice_code'])

    # 提取发票号码
    df.loc[index, 'invoice_num'] = re.search(r'发票号码.(\d+)', text).group(1)
    print("发票号码:", df.loc[index, 'invoice_num'])

    # 提取发票开具日期
    df.loc[index, 'invoice_date'] = re.search(r'开票日期:(\d+年\d+月\d+日)', text).group(1)
    print("开票日期:", df.loc[index, 'invoice_date'])

    # 提取购买方名称,销售方名称
    company_name = re.findall(r'名称:(\D.+?[司店龙馆局])', text)
    print(company_name)
    df.loc[index, 'buyer_name'] = company_name[0]
    print("购买方名称:", df.loc[index, 'buyer_name'])
    df.loc[index, 'seller_name'] = company_name[-1]
    print("销售方名称:", df.loc[index, 'seller_name'])


    # 提取购买方纳税人识别号
    buyer_tax_id = re.search(r'(?<=[\D])9[A-Z0-9]{17}', text)
    print(buyer_tax_id)
    df.loc[index, 'buyer_tax_id'] = buyer_tax_id[0]
    print("购买方纳税人识别号:", df.loc[index, 'buyer_tax_id'])


    # 提取服务名称
    df.loc[index, 'service_name'] = re.search(r'(?<=税额).*?(?:费|服务)', text).group()
    print("服务名称:", df.loc[index, 'service_name'])

    # 提取价税合计金额
    total_price = re.findall(r'[¥￥]\d+(?:\.\d+)?', text)
    df.loc[index, 'total_price']= total_price[-1]
    print("价税合计:", total_price[-1])
    
    print("#"*50)

文件名 2023年09月22日福州世茂实业有限公司-住宿服务52348402---¥760.56.pdf
发票号码: 52348402
开票日期: 2023年09月22日
['杭州趣链科技有限公司', '福州世茂实业有限公司']
购买方名称: 杭州趣链科技有限公司
销售方名称: 福州世茂实业有限公司
<re.Match object; span=(169, 187), match='91330108MA27Y5XH5G'>
购买方纳税人识别号: 91330108MA27Y5XH5G
服务名称: *住宿服务
价税合计: ¥760.56
##################################################
文件名 2023年09月27日杭州趣链科技有限公司-餐饮服务42272580---¥304.00.pdf
发票号码: 42272580
开票日期: 2023年09月27日
['杭州趣链科技有限公司']
购买方名称: 杭州趣链科技有限公司
销售方名称: 杭州趣链科技有限公司
<re.Match object; span=(168, 186), match='91330108MA27Y5XH5G'>
购买方纳税人识别号: 91330108MA27Y5XH5G
服务名称: *餐饮服务
价税合计: ¥304.00
##################################################
文件名 2023年10月16日阿斯兰航空服务（上海）有限公司-经纪代理服务75784642---￥830.00.pdf
发票号码: 75784642
开票日期: 2023年10月16日
['杭州趣链科技有限公司', '阿斯兰航空服务（上海）有限公司']
购买方名称: 杭州趣链科技有限公司
销售方名称: 阿斯兰航空服务（上海）有限公司
<re.Match object; span=(148, 166), match='91330108MA27Y5XH5G'>
购买方纳税人识别号: 91330108MA27Y5XH5G
服务名称: *经纪代理服务
价税合计: ￥830.00
##################################################
文件名 2023年10月18日厦门麦召餐饮有限公司-餐饮服务23

C:\Users\laoxu\AppData\Local\Temp\ipykernel_9084\222300330.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '52348402' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'invoice_num'] = re.search(r'发票号码.(\d+)', text).group(1)
C:\Users\laoxu\AppData\Local\Temp\ipykernel_9084\222300330.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023年09月22日' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'invoice_date'] = re.search(r'开票日期:(\d+年\d+月\d+日)', text).group(1)
C:\Users\laoxu\AppData\Local\Temp\ipykernel_9084\222300330.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '杭州趣链科技有限公司' has dtype incompatible with float64, please explicitly cast to a compatible dty

In [22]:
# 遍历 filename 列的每一行，并将文件重命名为 1+2+3.pdf 的格式
for index, row in df.iterrows():
    old_name = row['filename']
    new_name = row['invoice_date'] + row['seller_name'] + row['service_name'] + row['invoice_num'] + "---" +row['total_price'] + '.pdf'
    new_name = new_name.replace('*','-')
    if not os.path.exists(directory + new_name):
        os.rename(directory + old_name,directory + new_name)
        print(index,'o:',old_name,'===>>>',new_name,'\n')
    else:
        print(index,'x:',new_name,'已存在\n')


0 x: 2023年09月22日福州世茂实业有限公司-住宿服务52348402---¥760.56.pdf 已存在

1 x: 2023年09月27日杭州趣链科技有限公司-餐饮服务42272580---¥304.00.pdf 已存在

2 x: 2023年10月16日阿斯兰航空服务（上海）有限公司-经纪代理服务75784642---￥830.00.pdf 已存在

3 x: 2023年10月18日厦门麦召餐饮有限公司-餐饮服务23942000000020628958---¥534.00.pdf 已存在

4 x: 2023年10月20日华贤（厦门）酒店-住宿服务23942000000020792910---¥1394.02.pdf 已存在

5 x: 2023年10月26日长沙市雨花区张氏父子肠子馆-餐饮服务21045015---¥308.90.pdf 已存在

6 x: 2023年10月31日汉堡王食品（深圳）有限公司-餐饮服务05199405---¥107.00.pdf 已存在

7 x: 2023年11月12日华贤（厦门）酒店-住宿服务23942000000024220325---¥1423.98.pdf 已存在

8 x: 2023年11月13日厦门市思明区鹭岛假日快捷酒店-住宿服务23942000000024301019---¥770.00.pdf 已存在

9 x: 2023年11月15日恭子周衢味（广州）餐饮有限公司-餐饮服务23442000000283072055---¥601.00.pdf 已存在

10 x: 2023年11月20日元气寿司-餐饮服务82178597---¥231.00.pdf 已存在

11 x: 2023年11月21日太二广州星寰国际商业中心店-餐饮服务82463977---¥165.80.pdf 已存在

12 x: 2023年11月22日广州市金越轩餐饮管理有限公司-餐饮服务23442000000293516464---¥302.00.pdf 已存在

13 x: 2023年11月23日广州市客广星餐饮服务有限公司-餐饮服务23442000000295294149---¥286.00.pdf 已存在

14 x: 2023年11月24日广州灿幸晴餐饮管理有限公司-餐饮服务23442000000296195943---¥97